In [ ]:
# this is for eye lokk and yawn detection by model
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras

# Load Models
class_labels = {
    0: "forward_look",
    1: "close_look",
    2: "right_look",
    3: "left_look"
}

yawn_labels = {
    0: 'yawn',
    1: 'no_yawn'
}

try:
    eye_model = keras.models.load_model("eye_model.h5")
    print(eye_model.summary()) 
    yawn_model = keras.models.load_model("yawn_modal.h5")
    print("✅ Model loaded successfully!")
except Exception as e:
    print(f"❌ Error loading model: {e}")

IMG_SIZE = (120, 120)  

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_eye.xml")

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
    eyes = eye_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5, minSize=(60, 60))
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5, minSize=(80, 80))
    
    predictions_text = []

    if len(eyes) > 0:  # Ensure eyes are detected
        for (x, y, w, h) in eyes[:2]:  # Process only the first two detected eyes
            eye = frame[y:y+h, x:x+w]  # Extract eye region
            eye = cv2.resize(eye, IMG_SIZE)  
            eye = eye.astype("float32") / 255.0  
            eye = np.expand_dims(eye, axis=0)  

            predictions = eye_model.predict(eye)
            predicted_index = np.argmax(predictions)
            confidence = np.max(predictions)
            predicted_label = class_labels.get(predicted_index, "Unknown")

            predictions_text.append(f"{predicted_label} ({confidence:.2f})")

            cv2.putText(frame, f"{predicted_label} ({confidence:.2f})",
                        (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    if len(faces) > 0:  # Ensure faces are detected
        for (x, y, w, h) in faces:
            face_roi = frame[y:y+h+10, x:x+w+10]  # Extract face region
            face_roi = cv2.resize(face_roi, (145, 145))
            face_roi = face_roi.astype("float32") / 255.0
            face_roi = np.expand_dims(face_roi, axis=0)

            yawn_prediction = yawn_model.predict(face_roi)
            yawn_index = np.argmax(yawn_prediction)
            yawn_label = yawn_labels[yawn_index]

            cv2.putText(frame, f"Yawn: {yawn_label}",
                        (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
            cv2.rectangle(frame, (x, y), (x + w+5, y + h+15), (0, 0, 255), 2)

    if predictions_text:
        text = " | ".join(predictions_text)
        text_size = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)[0]
        text_x = (frame.shape[1] - text_size[0]) // 2  # Center horizontally
        text_y = 30  # Position near the top

        cv2.putText(frame, text, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)


    cv2.imshow("Real-Time Eye & Yawn Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)               │ (None, 120, 120, 16)   │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 120, 120, 16)   │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 60, 60, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 60, 60, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 60, 60, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 30, 30, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 30, 30, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 30, 30, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 15, 15, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 15, 15, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 15, 15, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 7, 7, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 6272)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 120)            │       752,760 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 120)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 4)              │           484 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 851,646 (3.25 MB)

 Trainable params: 851,164 (3.25 MB)

 Non-trainable params: 480 (1.88 KB)

 Optimizer params: 2 (12.00 B)

None


✅ Model loaded successfully!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━

In [ ]:
# this is only for eye aspect ratio 
import cv2
import mediapipe as mp
import numpy as np

mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(refine_landmarks=True)
mp_drawing = mp.solutions.drawing_utils

LEFT_EYE = [33, 160, 158, 133, 153, 144]
RIGHT_EYE = [362, 385, 387, 263, 373, 380]

FULLY_OPEN_THRESHOLD = 0.25
HALF_OPEN_THRESHOLD = 0.18
CLOSED_THRESHOLD = 0.12

def calculate_ear(eye_landmarks):
    p1, p2, p3, p4, p5, p6 = eye_landmarks
    vertical_1 = np.linalg.norm(np.array(p2) - np.array(p6))
    vertical_2 = np.linalg.norm(np.array(p3) - np.array(p5))
    horizontal = np.linalg.norm(np.array(p1) - np.array(p4))
    ear = (vertical_1 + vertical_2) / (2.0 * horizontal)
    return ear

def get_eye_landmarks(landmarks, eye_indices, frame_width, frame_height):
    return [(int(landmarks[idx].x * frame_width), int(landmarks[idx].y * frame_height)) for idx in eye_indices]

cap = cv2.VideoCapture(0)

if not cap.isOpened():
    exit()

while True:
    ret, frame = cap.read()
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(frame_rgb)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            left_eye_points = get_eye_landmarks(face_landmarks.landmark, LEFT_EYE, frame.shape[1], frame.shape[0])
            right_eye_points = get_eye_landmarks(face_landmarks.landmark, RIGHT_EYE, frame.shape[1], frame.shape[0])

            left_ear = calculate_ear(left_eye_points)
            right_ear = calculate_ear(right_eye_points)
            avg_ear = (left_ear + right_ear) / 2.0

            if avg_ear > FULLY_OPEN_THRESHOLD:
                eye_status = "Fully Opened"
            elif avg_ear > HALF_OPEN_THRESHOLD:
                eye_status = "Half Opened"
            else:
                eye_status = "Closed"

            cv2.putText(frame, f"EAR: {avg_ear:.2f} - {eye_status}", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0) if eye_status == "Fully Opened" else (0, 0, 255), 2)

    cv2.imshow('Eye Aspect Ratio Detection', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
# this is foe head position nd yawn detection 

import cv2
import mediapipe as mp
import pandas as pd
import time
import numpy as np

# Initialize Mediapipe FaceMesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(refine_landmarks=True)


MOUTH_INDICES = [61, 67, 62, 66, 63, 65, 60, 64] 
NOSE_INDEX = 1 
LEFT_EYE_INDEX = 33  # Left eye corner
RIGHT_EYE_INDEX = 263  # Right eye corner

def calculate_mar(landmarks):
    A = np.linalg.norm(landmarks[1] - landmarks[5])  # Upper-Lower Lip Distance
    B = np.linalg.norm(landmarks[2] - landmarks[4])
    C = np.linalg.norm(landmarks[0] - landmarks[3])  # Horizontal Distance
    MAR = (A + B) / (2.0 * C)
    return MAR

def head_pose(landmarks):
    nose = landmarks[NOSE_INDEX]
    left_eye = landmarks[LEFT_EYE_INDEX]
    right_eye = landmarks[RIGHT_EYE_INDEX]

    # Calculate average eye height for reference
    eye_avg_y = (left_eye[1] + right_eye[1]) / 2

    # Head turned left or right
    if nose[0] < left_eye[0] - 5:
        return "Right Look"
    elif nose[0] > right_eye[0] + 5:
        return "Left Look"
    if nose[1] > eye_avg_y+60: 
        return "Down Look"

    return "Forward Look" 


# Initialize Data Table
data = []

# Start Video Capture
cap = cv2.VideoCapture(0)
start_time = time.time()

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    frame = cv2.flip(frame, 1)  # Mirror Effect
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb_frame)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            landmarks = [(int(l.x * frame.shape[1]), int(l.y * frame.shape[0])) for l in face_landmarks.landmark]

            mouth_points = np.array([landmarks[i] for i in MOUTH_INDICES])
            mar = calculate_mar(mouth_points)
            yawn_status = "No Yawn" if mar > 0.65 else "Yawn"  # Adjust threshold

            head_status = head_pose(landmarks)

            text_position = (30, 50)  # Top-left
            cv2.putText(frame, f"Yawn: {yawn_status} | Look: {head_status}", text_position, 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

            if time.time() - start_time >= 1:
                data.append([yawn_status, head_status])
                start_time = time.time()


    cv2.imshow("Concentration Monitor", frame)

    # Quit on 'q' key
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# Convert to DataFrame
df = pd.DataFrame(data, columns=["Yawn Status", "Head Pose"])

# Save as CSV
df.to_csv("concentration_data.csv", index=False)

print("Data Saved!")
print(df)


Data Saved!
   Yawn Status     Head Pose
0         Yawn  Forward Look
1         Yawn  Forward Look
2         Yawn  Forward Look
3         Yawn  Forward Look
4         Yawn  Forward Look
5      No Yawn  Forward Look
6      No Yawn  Forward Look
7      No Yawn  Forward Look
8      No Yawn  Forward Look
9      No Yawn  Forward Look
10     No Yawn  Forward Look
11        Yawn  Forward Look
12     No Yawn  Forward Look
13     No Yawn  Forward Look
14     No Yawn  Forward Look
15     No Yawn  Forward Look
16     No Yawn  Forward Look
17     No Yawn  Forward Look
18     No Yawn  Forward Look
19     No Yawn  Forward Look
20     No Yawn  Forward Look
21     No Yawn  Forward Look
22     No Yawn  Forward Look
23     No Yawn  Forward Look
24     No Yawn  Forward Look
25     No Yawn  Forward Look
26     No Yawn  Forward Look
27     No Yawn  Forward Look
28     No Yawn  Forward Look
29     No Yawn  Forward Look
30     No Yawn  Forward Look
31     No Yawn  Forward Look
32     No Yawn  Forward Look
33

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import time
import tensorflow as tf
from tensorflow import keras

# Initialize Mediapipe FaceMesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(refine_landmarks=True)
mp_drawing = mp.solutions.drawing_utils

# Facial for Eyes, Mouth, and Nose
LEFT_EYE = [33, 160, 158, 133, 153, 144]
RIGHT_EYE = [362, 385, 387, 263, 373, 380]
MOUTH_INDICES = [61, 67, 62, 66, 63, 65, 60, 64]
NOSE_INDEX = 1
LEFT_EYE_INDEX = 33
RIGHT_EYE_INDEX = 263

# Eye Aspect Ratio (EAR) Thresholds
FULLY_OPEN_THRESHOLD = 0.25
HALF_OPEN_THRESHOLD = 0.18
CLOSED_THRESHOLD = 0.12

# Initialize data table
data = []
start_time = time.time()

#lables of eye lokkings
class_labels = {
    0: "forward_look",
    1: "close_look",
    2: "right_look",
    3: "left_look"
}

#for eyes modal loading 

try:
    eye_model = keras.models.load_model("eye_model.h5")
    print("Model loaded successfully!")
except Exception as e:
    print(f" Error loading model: {e}")

IMG_SIZE = (120, 120)  

eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_eye.xml")

# Calculate Eye Aspect Ratio (ear)
def calculate_ear(eye_landmarks):
    p1, p2, p3, p4, p5, p6 = eye_landmarks
    vertical_1 = np.linalg.norm(np.array(p2) - np.array(p6))
    vertical_2 = np.linalg.norm(np.array(p3) - np.array(p5))
    horizontal = np.linalg.norm(np.array(p1) - np.array(p4))
    ear = (vertical_1 + vertical_2) / (2.0 * horizontal)
    return ear

# Extract Eye Landmarks
def get_eye_landmarks(landmarks, eye_indices, frame_width, frame_height):
    return [(int(landmarks[idx].x * frame_width), int(landmarks[idx].y * frame_height)) for idx in eye_indices]

# Calculate Mouth Aspect Ratio (MAR) for yawning detection
def calculate_mar(landmarks):
    A = np.linalg.norm(landmarks[1] - landmarks[5])  # Upper-Lower Lip Distance
    B = np.linalg.norm(landmarks[2] - landmarks[4])
    C = np.linalg.norm(landmarks[0] - landmarks[3])  # Horizontal Distance
    MAR = (A + B) / (2.0 * C)
    return MAR

# Detect Head Pose
def head_pose(landmarks):
    nose = landmarks[NOSE_INDEX]
    left_eye = landmarks[LEFT_EYE_INDEX]
    right_eye = landmarks[RIGHT_EYE_INDEX]

    eye_avg_y = (left_eye[1] + right_eye[1]) / 2

    if nose[0] < left_eye[0] - 5:
        return "Right Look"
    elif nose[0] > right_eye[0] + 5:
        return "Left Look"
    if nose[1] > eye_avg_y + 60:
        return "Down Look"

    return "Forward Look"

# Start Video Capture
cap = cv2.VideoCapture(0)

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    frame = cv2.flip(frame, 1)  # Mirror Effect
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb_frame)
    
    eyes = eye_cascade.detectMultiScale(rgb_frame, scaleFactor=1.3, minNeighbors=5, minSize=(60, 60))
    
    predictions_text = []

    if len(eyes) > 0: 
        for (x, y, w, h) in eyes[:2]:  
            eye = frame[y:y+h, x:x+w] 
            eye = cv2.resize(eye, IMG_SIZE)  
            eye = eye.astype("float32") / 255.0  
            eye = np.expand_dims(eye, axis=0)  

            predictions = eye_model.predict(eye)
            predicted_index = np.argmax(predictions)
            confidence = np.max(predictions)
            predicted_label = class_labels.get(predicted_index, "Unknown")

            predictions_text.append(f"{predicted_label}")

            cv2.putText(frame, f"{predicted_label}",
                        (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 255, 0), 1)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
    
    if predictions_text:
        text = " | ".join(predictions_text)
        text_size = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)[0]
        text_x = (frame.shape[0] - text_size[1]) //2
        text_y = 100 

        cv2.putText(frame, text, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)


    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            landmarks = [(int(l.x * frame.shape[1]), int(l.y * frame.shape[0])) for l in face_landmarks.landmark]

            left_eye_points = get_eye_landmarks(face_landmarks.landmark, LEFT_EYE, frame.shape[1], frame.shape[0])
            right_eye_points = get_eye_landmarks(face_landmarks.landmark, RIGHT_EYE, frame.shape[1], frame.shape[0])

            left_ear = calculate_ear(left_eye_points)
            right_ear = calculate_ear(right_eye_points)
            avg_ear = (left_ear + right_ear) / 2.0

            if avg_ear > FULLY_OPEN_THRESHOLD:
                eye_status = "Fully Opened"
            elif avg_ear > HALF_OPEN_THRESHOLD:
                eye_status = "Half Opened"
            else:
                eye_status = "Closed"

            mouth_points = np.array([landmarks[i] for i in MOUTH_INDICES])
            mar = calculate_mar(mouth_points)
            yawn_status = "No Yawn" if mar > 0.65 else "Yawn"  # Adjust threshold

            head_status = head_pose(landmarks)

            cv2.putText(frame, f"EAR: {avg_ear:.2f} - {eye_status}", (10, 30), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            cv2.putText(frame, f"Yawn: {yawn_status} | Look: {head_status}", (10, 60), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)

            if time.time() - start_time >= 1:
                left_eye_pred = predictions_text[0] if len(predictions_text) > 0 else "Unknown"
                right_eye_pred = predictions_text[1] if len(predictions_text) > 1 else "Unknown"
                data.append([eye_status,left_eye_pred,right_eye_pred, yawn_status, head_status])
                start_time = time.time()


    cv2.imshow("Concentration Monitor", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# Convert to DataFrame
df = pd.DataFrame(data, columns=["Eye Status","left eye looking ","right eye looking", "Yawn Status", "Head Pose"])

# Save as CSV
df.to_csv("concentration_data.csv", index=False)

print("Data Saved!")
print(df)



Model loaded successfully!
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 522ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━

In [1]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import pandas as pd
# [eye_open,left_eye,right_eye,yawn_status,head_status]
data = pd.read_csv('concentration_data.csv')
print(data)

'''
X = np.array([
    [1, 1, 0, 1, 1],  # 100% concentration
    [0, 1, 1, 1, 1],  # Yawning - Low concentration
    [1, 0, 1, 1, 1],  # One eye closed - Low concentration
    [1, 1, 0, 1, 1],  # One eye closed - Low concentration
    [1, 1, 1, 0, 1],  # Not looking forward - Low concentration
    [0, 0, 0, 0, 1],  # Fully distracted
    [0, 1, 0, 0, 1],  # Partial distraction
    [1, 0, 0, 1, 1],  # Eyes not fully open
])

# labels 1 = 100% Concentration, 0 = Low Concentration
y = np.array([1, 0, 0, 0, 0, 0, 0, 0])


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = LogisticRegression()
model.fit(X_train, y_train)


new_sample = np.array([[1, 1, 1, 1,0]])  
prediction = model.predict(new_sample)
print("Concentration Level:", "100%" if prediction[0] == 1 else "Low")'
'''


      Eye Status left eye looking  right eye looking Yawn Status     Head Pose
0   Fully Opened      forward_look        right_look     No Yawn     Down Look
1   Fully Opened        right_look        right_look     No Yawn     Down Look
2   Fully Opened        right_look        right_look     No Yawn  Forward Look
3   Fully Opened      forward_look        right_look     No Yawn     Down Look
4   Fully Opened        right_look        right_look     No Yawn  Forward Look
5   Fully Opened        right_look        right_look     No Yawn     Down Look
6   Fully Opened        right_look           Unknown     No Yawn  Forward Look
7   Fully Opened        right_look        right_look     No Yawn     Down Look
8   Fully Opened      forward_look        right_look     No Yawn     Down Look
9   Fully Opened        right_look           Unknown     No Yawn  Forward Look
10  Fully Opened        right_look        right_look     No Yawn  Forward Look
11  Fully Opened        right_look        right_look

'\nX = np.array([\n    [1, 1, 0, 1, 1],  # 100% concentration\n    [0, 1, 1, 1, 1],  # Yawning - Low concentration\n    [1, 0, 1, 1, 1],  # One eye closed - Low concentration\n    [1, 1, 0, 1, 1],  # One eye closed - Low concentration\n    [1, 1, 1, 0, 1],  # Not looking forward - Low concentration\n    [0, 0, 0, 0, 1],  # Fully distracted\n    [0, 1, 0, 0, 1],  # Partial distraction\n    [1, 0, 0, 1, 1],  # Eyes not fully open\n])\n\n# labels 1 = 100% Concentration, 0 = Low Concentration\ny = np.array([1, 0, 0, 0, 0, 0, 0, 0])\n\n\nX_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)\nmodel = LogisticRegression()\nmodel.fit(X_train, y_train)\n\n\nnew_sample = np.array([[1, 1, 1, 1,0]])  \nprediction = model.predict(new_sample)\nprint("Concentration Level:", "100%" if prediction[0] == 1 else "Low")\'\n'

In [ ]:
#this is for eye gazing detection 
import cv2
import mediapipe as mp
import numpy as np

# Initialize Mediapipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, refine_landmarks=True)

# Define landmark indexes for eyes
LEFT_EYE = [33, 160, 158, 133, 153, 144]
RIGHT_EYE = [263, 387, 385, 362, 373, 380]

def get_eye_position_ratio(landmarks, frame_width, frame_height):
    """Calculate normalized eye position ratios for gaze detection."""
    left_eye_center = np.mean([landmarks[idx] for idx in LEFT_EYE], axis=0)
    right_eye_center = np.mean([landmarks[idx] for idx in RIGHT_EYE], axis=0)

    # Normalize eye positions relative to face size
    eye_x_ratio = (left_eye_center[0] + right_eye_center[0]) / (2 * frame_width)  # Horizontal ratio
    eye_y_ratio = (left_eye_center[1] + right_eye_center[1]) / (2 * frame_height) # Vertical ratio

    return eye_x_ratio, eye_y_ratio

def detect_gaze(eye_x_ratio, eye_y_ratio):
    """Detect gaze direction based on eye position ratio."""
    gaze_direction = "Center"

    if eye_x_ratio < 0.4:
        gaze_direction = "Looking Left"
    elif eye_x_ratio > 0.6:
        gaze_direction = "Looking Right"

    if eye_y_ratio < 0.4:
        gaze_direction += " - Up"
    elif eye_y_ratio > 0.6:
        gaze_direction += " - Down"

    return gaze_direction

# Start Video Capture
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)  # Mirror the frame
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame_height, frame_width, _ = frame.shape

    results = face_mesh.process(rgb_frame)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            # Extract landmark positions
            landmarks = [(int(landmark.x * frame_width), int(landmark.y * frame_height)) 
                         for landmark in face_landmarks.landmark]

            # Compute eye ratios
            eye_x, eye_y = get_eye_position_ratio(landmarks, frame_width, frame_height)

            # Detect gaze direction
            gaze_direction = detect_gaze(eye_x, eye_y)

            # Draw eye points
            for idx in LEFT_EYE + RIGHT_EYE:
                cv2.circle(frame, landmarks[idx], 2, (0, 255, 0), -1)

            # Display gaze direction
            cv2.putText(frame, f"Gaze: {gaze_direction}", (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    cv2.imshow("Eye Gaze Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
